## Homework 5: Weather Prediction

### Jacqueline Beechert, jbeechert@berkeley.edu

Due Tuesday March 8, 2022 @ 5pm

## 1. Use the data supplied of airport codes (ICAO_airports.csv) and top airports (top_airports.csv) to make a table of 50 most travelled airports in the US and containing relevant information, like name, wikipedia link, latitude, longitude

In [264]:
import pandas as pd
import numpy as np
import sqlite3

In [13]:
icao_df = pd.read_csv("hw_5_data/ICAO_airports.csv")
top_df = pd.read_csv("hw_5_data/top_airports.csv")

In [350]:
icao_df.head(5)

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
2,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
3,6526,00AR,heliport,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,00AR,NaN,00AR,NaN,NaN,NaN
4,6527,00AZ,small_airport,Cordes Airport,34.305599,-112.165001,3810.0,NaN,US,US-AZ,Cordes,no,00AZ,NaN,00AZ,NaN,NaN,NaN


In [445]:
top_50_df["Airport"].values

array(['Hartsfield-Jackson Atlanta International Airport',
       "Chicago O'Hare International Airport",
       'Los Angeles International Airport',
       'Dallas/Fort Worth International Airport',
       'Denver International Airport',
       'John F. Kennedy International Airport',
       'San Francisco International Airport',
       'George Bush Intercontinental Airport',
       'McCarran International Airport',
       'Phoenix Sky Harbor International Airport',
       'Charlotte/Douglas International Airport',
       'Miami International Airport', 'Orlando International Airport',
       'Newark Liberty International Airport',
       'Detroit Metropolitan Wayne County Airport',
       'Minneapolis-St. Paul International Airport (Wold-Chamberlain Field)',
       'Seattle-Tacoma International Airport',
       'Philadelphia International Airport',
       'Gen. Edward Lawrence Logan International Airport',
       'LaGuardia Airport (and Marine Air Terminal)',
       'Washington Dulles

In [451]:
top_50_df = top_df.nlargest(50, "Enplanements")
top_50_airport_names = [x.strip(" Airport") for x in top_50_df["Airport"].values]
print(top_50_airport_names)
top_50_airport_names[1] = 'Chicago O''Hare International'
print(top_50_airport_names)

['Hartsfield-Jackson Atlanta International', "Chicago O'Hare International", 'Los Angeles International', 'Dallas/Fort Worth International', 'Denver International', 'John F. Kennedy International', 'San Francisco International', 'George Bush Intercontinental', 'McCarran International', 'Phoenix Sky Harbor International', 'Charlotte/Douglas International', 'Miami International', 'Orlando International', 'Newark Liberty International', 'Detroit Metropolitan Wayne County', 'Minneapolis-St. Paul International Airport (Wold-Chamberlain Field)', 'Seattle-Tacoma International', 'Philadelphia International', 'Gen. Edward Lawrence Logan International', 'LaGuardia Airport (and Marine Air Terminal)', 'Washington Dulles International', 'Baltimore/Washington International Thurgood Marshall', 'Fort Lauderdale–Hollywood International', 'Salt Lake City International', 'Honolulu International Airport / Hickam AFB', 'Ronald Reagan Washington National', 'Chicago Midway International', 'San Diego Internat

In [452]:
# Silly check of if we uniquely associated one airport in top_50_df with one airport in icao_df
# This might be a good way to compare the tables later
l = []
for name in top_50_airport_names:
    l.append(icao_df.loc[icao_df["name"].str.contains(name)])

# Should be 50
print(f"Is this length 50? {len(l)==50}") 

/var/folders/b5/08p_fml511n2rn57cysbxlq80000gn/T/ipykernel_75684/1949688473.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  l.append(icao_df.loc[icao_df["name"].str.contains(name)])


Is this length 50? True


In [453]:
# make icao table
connection = sqlite3.connect(":memory:")

cursor = connection.cursor()

sql_cmd = """CREATE TABLE icao (id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT, wikipedia_link TEXT, latitude FLOAT, longitude FLOAT)"""

cursor.execute(sql_cmd)

# looping over top_50_airport_names with this if statement is very slow
#for top_name in top_50_airport_names:

for i, row in icao_df.iterrows():

    icao_name = row['name']
    wiki = str(row["wikipedia_link"])
    latitude = row["latitude_deg"]
    longitude = row["longitude_deg"]    

    data = (icao_name, wiki, latitude, longitude)

    # looping over top_50_airport_names with this if statement is very slow
    #if icao_name in top_name:
    #    print(icao_name, top_name)

    sql_cmd = ("INSERT INTO icao (name, wikipedia_link, latitude, longitude) VALUES " + str(data))
    cursor.execute(sql_cmd)
    
connection.commit()
#connection.close()    

In [311]:
# cursor.execute("""SELECT * from icao""")
# icao_info = cursor.fetchall()
# for entry in icao_info: 
#     print(entry)

In [429]:
# # make top 50 table
# #connection = sqlite3.connect(":memory:")

# cursor = connection.cursor()

# sql_cmd = """CREATE TABLE top50 (id INTEGER PRIMARY KEY AUTOINCREMENT,
#     city TEXT, airport TEXT, enplanements INT)"""

# cursor.execute(sql_cmd)

# for i, row in top_50_df.iterrows():
    
#     city = row['City']
#     airport = str(row["Airport"])
#     enplanements = row["Enplanements"]
    
#     data = (city, airport, enplanements)
    
#     sql_cmd = ("INSERT INTO top50 (city, airport, enplanements) VALUES " + str(data))
#     cursor.execute(sql_cmd)
    
# connection.commit()

In [457]:
print(top_50_airport_names[:5],"\n")
for top_name in top_50_airport_names:

    #sql_cmd = f"""SELECT * FROM icao WHERE name LIKE '{top_name}'""" 
    sql_cmd = f"""SELECT * FROM icao WHERE CONTAINS(name, '{top_name}')""" 
    
    print(sql_cmd)
    cursor.execute(sql_cmd)

    info = cursor.fetchall()
    print(info)

['Hartsfield-Jackson Atlanta International', 'Chicago OHare International', 'Los Angeles International', 'Dallas/Fort Worth International', 'Denver International'] 

SELECT * FROM icao WHERE CONTAINS(name, 'Hartsfield-Jackson Atlanta International')


OperationalError: no such function: CONTAINS

In [399]:
info = cursor.fetchall()

#sql_cmd = """CREATE TABLE icao_top50 (id INTEGER PRIMARY KEY AUTOINCREMENT,
#    name TEXT, wikipedia_link TEXT, latitude FLOAT, longitude FLOAT)"""
#cursor.execute(sql_cmd)

for entry in info: 
    print(entry)

(21574, 'Dallas Love Field', 'http://en.wikipedia.org/wiki/Dallas_Love_Field', 32.8470993041992, -96.8517990112305)


## 2. Build another table that will hold historical weather information: min/max temperature, relative humidity, and precipitation

In [ ]:
# make weather table
connection = sqlite3.connect(":memory:")

cursor = connection.cursor()

sql_cmd = """CREATE TABLE weather (id INTEGER PRIMARY KEY AUTOINCREMENT,
    min_temp FLOAT, max_temp FLOAT, rel_humidity FLOAT, precipitation FLOAT)"""

cursor.execute(sql_cmd)

## 3. Using the data from the xarray/netCDF4 example in class, populate the database from 1990-2000

In [189]:
import xarray as xr

In [252]:
# Data path from lecture
max_temp_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_tasmax_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

# Open the data set as xarray
max_temp_xr  = xr.open_dataset(max_temp_path)  

# Select data between 1990-2000
max_temp_xr = max_temp_xr.sel(time=slice('1990', '2000')).load()

# View xarray object
#max_temp_xr   # data variable: air_temperature

In [233]:
# maximum/minimum temperature(tasmax/tasmin)
# maximum/minimum relative humidity (rhsmax/rhsmin)
# precipitation amount(pr)

In [228]:
min_temp_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_tasmin_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

# Open the data set as xarray
min_temp_xr  = xr.open_dataset(min_temp_path)  

# Select data between 1990-2000
min_temp_xr = min_temp_xr.sel(time=slice('1990', '2000')).load()

# View xarray object
#min_temp_xr   # data variable: air_temperature

In [230]:
max_hum_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_rhsmax_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

# Open the data set as xarray
max_hum_xr  = xr.open_dataset(max_hum_path)  

# Select data between 1990-2000
max_hum_xr = max_hum_xr.sel(time=slice('1990', '2000')).load()

In [239]:
#max_hum_xr  # data variable: relative_humidity

In [232]:
min_hum_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_rhsmin_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

# Open the data set as xarray
min_hum_xr  = xr.open_dataset(min_hum_path)  

# Select data between 1990-2000
min_hum_xr = min_hum_xr.sel(time=slice('1990', '2000')).load()

In [237]:
#min_hum_xr   # data variable: relative_humidity

In [231]:
precip_path = "http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_macav2metdata_pr_BNU-ESM_r1i1p1_historical_1950_2005_CONUS_monthly.nc" 

# Open the data set as xarray
precip_xr  = xr.open_dataset(precip_path)  

# Select data between 1990-2000
precip_xr = precip_xr.sel(time=slice('1990', '2000')).load()

In [238]:
#precip_xr    # data variable: precipitation

In [306]:
lats = max_temp_xr["air_temperature"]["lat"].values
lons = max_temp_xr["air_temperature"]["lon"].values

latitudes = np.array([round(x,2) for x in lats])
longitudes = np.array([round(x,2) for x in lons])

lat_idx = list(np.where(latitudes == 40.1)[0])
lon_idx = list(np.where(longitudes == -74.9+360)[0])

In [305]:
one_point = max_temp_xr["air_temperature"].sel(lat=lats[lat_idx],
                                               lon=lons[lon_idx])
print((one_point.values).shape)

(132, 1, 1)
